In [2]:
import numpy as np
import pandas as pd 
import os
import sklearn
import nltk

# Data Preprocessing

In [ ]:
# чистка выбросов

import pandas as pd

# Example DataFrame
df = pd.DataFrame({'value': [12, 15, 14, 10, 102, 13, 11, 14, 13, 15, 110]})
# Compute IQR boundaries
Q1 = df['value'].quantile(0.25)
Q3 = df['value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers outside the IQR bounds
df_clean = df[~((df['value'] < lower_bound) | (df['value'] > upper_bound))]
print(df_clean)


#resampling
from imblearn.over_sampling import SMOTE

X_train, y_train = ...  # your feature matrix and labels for training
print("Class distribution before:", dict(pd.Series(y_train).value_counts()))

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)
print("Class distribution after:", dict(pd.Series(y_res).value_counts()))


# NLP

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)


# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)


import spacy
nlp = spacy.load("en_core_web_sm")

def spacy_preprocess(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]



In [1]:
from gensim.models import Word2Vec

sentences = [["i", "like", "nlp"], ["word", "embeddings", "are", "cool"]]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1)
vector = model.wv['nlp']
print(vector[:5])  # пример вектора


[-0.00713902  0.00124103 -0.00717672 -0.00224462  0.0037193 ]


In [ ]:
import numpy as np

def load_glove(file_path):
    embeddings = {}
    with open(file_path, encoding="utf8") as f:
        for line in f:
            parts = line.split()
            word = parts[0]
            vector = np.array(parts[1:], dtype="float32")
            embeddings[word] = vector
    return embeddings

glove = load_glove("glove.6B.100d.txt")
print(glove["king"][:5])


In [ ]:
from gensim.models import FastText

sentences = [["i", "like", "natural", "language", "processing"]]
model = FastText(sentences, vector_size=100, window=3, min_count=1)
print(model.wv["language"][:5])

#или

import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # скачивает модель

ft = fasttext.load_model('cc.en.300.bin')
print(ft.get_word_vector("hello")[:5])


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def nltk_lemmatize(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [t for t in tokens if t not in stop_words]


In [ ]:
# GRU with glove embeddings and two dense layers
from keras.layers import Dense, GRU, Dropout, Activation, Embedding, SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.model import Sequential

model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

In [ ]:
# размер вокаба

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
vocab_size = len(tokenizer.word_index) + 1

In [2]:
#полный пайплайн

import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer_en = WordNetLemmatizer()

stop_words_en = set(stopwords.words('english'))

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text) #числа
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[^a-z]', ' ', text)
    
    tokens = word_tokenize(text, language='english')
    
    tokens = [word for word in tokens if word not in stop_words_en and len(word) > 2]
    pos_tags = nltk.pos_tag(tokens)
    tokens = [lemmatizer_en.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    
    return tokens



[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


# Ensembling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = LogisticRegression()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = svm.SVC(probability=True)

ensemble = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('svm', clf3)], voting='hard')
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)